## 합성곱/풀링 계층 구현하기

4차원 배열

In [1]:
import numpy as np

In [3]:
x = np.random.rand(10, 1, 28, 28) # 데이터 수, 채널, 높이, 너비
print(x.shape) 
print(x.shape[0])
print(x.shape[1])

(10, 1, 28, 28)
10
1


im2col로 데이터 전개하기

In [ ]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h) // stride + 1
    out_w = (W + 2*pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') ## height, width에만 padding 추가
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
    
    ## N, out_h, out_w, C, h, w -> (N * out_h * out_w, C * h * w)
    col = col.transpose(0, 4, 5, 1, 2 ,3).reshape(N * out_h * out_w, -1)

In [8]:
input_data = np.random.rand(6, 3, 2, 2)
pad = 1
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')

In [13]:
import numpy as np

# 예시 데이터 설정
N, C, H, W = 1, 1, 4, 4  # 1개의 이미지, 1개의 채널, 4x4 크기
filter_h, filter_w, stride, pad = 2, 2, 2, 0  # 2x2 필터, 스트라이드 2, 패딩 0

# 4x4 크기의 이미지 데이터 생성 (값은 행과 열 인덱스의 합으로 설정)
input_data = np.array([[[[i*4 + j for j in range(W)] for i in range(H)]]])
print("원본 이미지 데이터:")
print(input_data)

# 출력 데이터의 높이와 너비 계산
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

# 패딩 적용 (이 경우에는 패딩이 0이므로 원본 데이터 그대로 사용)
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')

# 2차원 배열 초기화
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

# 이미지 데이터의 평탄화
for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

print("\n평탄화된 데이터 (col 배열):")
print(col)

# 차원 재배열 및 최종 2차원 배열 생성
col = col.transpose(0, 4, 5, 1, 2, 3)
print("\ntranspose 배열 (col 배열):")
print(col)

col = col.reshape(N*out_h*out_w, -1)

print("\n최종 2차원 배열 (col 배열):")
print(col)


원본 이미지 데이터:
[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]
   [12 13 14 15]]]]

평탄화된 데이터 (col 배열):
[[[[[[ 0.  2.]
     [ 8. 10.]]

    [[ 1.  3.]
     [ 9. 11.]]]


   [[[ 4.  6.]
     [12. 14.]]

    [[ 5.  7.]
     [13. 15.]]]]]]

transpose 배열 (col 배열):
[[[[[[ 0.  1.]
     [ 4.  5.]]]


   [[[ 2.  3.]
     [ 6.  7.]]]]



  [[[[ 8.  9.]
     [12. 13.]]]


   [[[10. 11.]
     [14. 15.]]]]]]

최종 2차원 배열 (col 배열):
[[ 0.  1.  4.  5.]
 [ 2.  3.  6.  7.]
 [ 8.  9. 12. 13.]
 [10. 11. 14. 15.]]
